In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

### 1. A bank found that the average number of cars waiting during the noon hour at a drive-up window follows a Poisson distribution with a mean of 2 cars. Make a chart of this distribution and answer these questions concerning the probability of cars waiting at the drive-up window.

In [2]:
rate = 2 # cars waiting at noon 

#### * What is the probability that no cars drive up in the noon hour?

In [3]:
stats.poisson(rate).pmf(0)

0.1353352832366127

#### * What is the probability that 3 or more cars come through the drive through?

In [21]:
stats.poisson(rate).sf(2)

0.32332358381693654

#### * How likely is it that the drive through gets at least 1 car?

In [5]:
stats.poisson(rate).cdf(1)

0.40600584970983794

### 2. Grades of State University graduates are normally distributed with a mean of 3.0 and a standard deviation of .3. Calculate the following:

In [6]:
mean = 3
std = .3
grades = stats.norm(mean, std)

#### * What grade point average is required to be in the top 5% of the graduating class?

In [7]:
grades.isf(.05)

3.4934560880854417

#### * What GPA constitutes the bottom 15% of the class?

In [8]:
grades.ppf(.15)

2.689069983151863

#### * An eccentric alumnus left scholarship money for students in the third decile from the bottom of their class. Determine the range of the third decile. Would a student with a 2.8 grade point average qualify for this scholarship?

In [9]:
lower_bound_third_decile = grades.ppf(.2)
upper_bound_third_decile = grades.ppf(.3)

print(f"{lower_bound_third_decile} - {upper_bound_third_decile}")
# A student with a 2.8 gpa would qualify for the scholarship

2.7475136299281258 - 2.842679846187588


#### * If I have a GPA of 3.5, what percentile am I in?

In [10]:
1 - grades.sf(3.5)
# ~95th percentile

0.9522096477271853

### 3. A marketing website has an average click-through rate of 2%. One day they observe 4326 visitors and 97 click-throughs. How likely is it that this many people or more click through?

In [11]:
probability = click_through_rate = .02
n_trials = visitors = 4326

stats.binom(n_trials, probability).sf(97)

0.11781530585907925

### 4. You are working on some statistics homework consisting of 100 questions where all of the answers are a probability rounded to the hundreths place. Looking to save time, you put down random probabilities as the answer to each question.

In [12]:
probability = chance_of_guessing = 1/100
n_trials = questions_answered = 60

#### * What is the probability that at least one of your first 60 answers is correct?

In [13]:
stats.binom(n_trials, probability).sf(1)

0.12123327131180733

### 5. The codeup staff tends to get upset when the student break area is not cleaned up. Suppose that there's a 3% chance that any one student cleans the break area when they visit it, and, on any given day, about 90% of the 3 active cohorts of 22 students visit the break area. How likely is it that the break area gets cleaned up each day?

In [18]:
probability = chance_of_cleaning = .03
n_trials = visitors = .9 * (22 * 3)

stats.binom(n_trials, probability).sf(0)

0.8342199288437355

#### * How likely is it that it goes two days without getting cleaned up?

In [19]:
outcomes = [0, 1] # 1 is cleaned
daily_chance_of_cleaning = stats.binom(n_trials, probability).sf(1)
n_trials = days_straight = 2
n_simulations = 100_000

sims = np.random.choice(outcomes, size = (n_simulations, n_trials), p = [1 - daily_chance_of_cleaning, daily_chance_of_cleaning])
no_cleaning_2 = sims.sum(axis = 1) == 0
no_cleaning_2.mean()

0.21769

#### * All week?

In [20]:
sims = np.random.choice(outcomes, size = (n_simulations, 7), p = [1 - daily_chance_of_cleaning, daily_chance_of_cleaning])

no_cleaning_7 = sims.sum(axis = 1) == 0
no_cleaning_7.mean()

0.00476